In [19]:
import tensorflow as tf

#Actions
* Move (up, right, down, left)
* Rotation
* Jump

#Observations
> camera input with following legend
* black => background
* green => goal
* red => obstacles
* white => walls

#Reward
* hit goal -> +10
* hit obstacle -> -1
* time's out -> -10

#Episodes
>Start => will Spawn agent and start timer
>End => due to time's out or hit goal

In [20]:
# Hyperparameters
camera_shape = (100, 100)
input_shape = camera_shape[0] * camera_shape[1]
num_actions = 4 + 2 + 1 # 4 directions, 2 rotate, 1 jump

# Model
learning_rate = 0.0001

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)), # Input layer
    tf.keras.layers.Dense(64, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(num_actions) # Output layer with num_actions neurons
])

In [22]:
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) # Adjust the learning rate as needed
model.compile(optimizer=optimizer, loss=custom_loss)